In [1]:
import torch
import os
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import pandas as pd

from model.classifier import Net as ClassNet
from torch.utils.data import Dataset, TensorDataset, DataLoader

trained_class = '../result/gpu_classification_g3train.pt'
class_model = ClassNet()
class_model.load_state_dict(torch.load(trained_class, map_location=torch.device('cuda')))
class_model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class_model = class_model.double()
class_model.to(device)

print("Model Loaded")

print("Importing Data")

/tmpscratch/tkim12/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model Loaded
Importing Data


In [2]:
# Implant PHS information

num_HS = 1
g_rescale = 3

In [ ]:
# Load Sky map
path="/afs/crc.nd.edu/user/t/tkim12/Work/CMB_ML/Data/Nside1024/quicklens_720x720/"
pathsig = '/afs/crc.nd.edu/user/t/tkim12/Work/CMB_ML/Data/Nside1024/test_img/'

#bkg_only = np.empty((0, 720, 720))
sig_only = np.empty((0, 720, 720))

#bkg_number = 1

for bkg_number in range(2,3):
    
    bkg_only = np.load(path+"500_events_720Sqr_lensed_uncutBackgroundlmax3500_"+str(bkg_number)+".npy")


    for evnts in range(8):
        temp2 = np.load(pathsig+"sig_larger/200_PHS_720sq_g1_Sig_onLSS_"+str(evnts+1)+".npy")
        sig_only = np.concatenate((sig_only,temp2))

    steps = 5

    Xlowermax = int((bkg_only[0].T.shape[0]-90)/steps)*steps
    Ylowermax = int((bkg_only[0].T.shape[1]-90)/steps)*steps

    print(bkg_only[0].shape)
    print(Xlowermax,Ylowermax)

    all_pred = []
    sampled_sig = []

    for evnt_num in range(bkg_only.shape[0]):
    #for evnt_num in range(2):

        print("\n\nEvent number = "+str(evnt_num+1)+"\n")

        if g_rescale != 0:
            sampling = torch.randperm(len(sig_only))[:num_HS].numpy()
            sampled_sig.append(sampling)

            sigs = np.zeros((bkg_only[0].shape[0],bkg_only[0].shape[1]))

            for ii in range(len(sampling)):
                sigs += sig_only[sampling[ii]]

            tot = bkg_only[evnt_num]+g_rescale*sigs

        elif g_rescale == 0:
            tot = bkg_only[evnt_num]

        i=0
        pred_outputs = []
        pred_box_loc = []
        pred_box_val = []
        boundary_box = []

        temp_pred = []
        X_data = []
        Y_data = []

        for xlower in range(int(Xlowermax/steps+1)):

            for ylower in range(int(Ylowermax/steps+1)):

                temp = (tot)[xlower*steps:(xlower*steps+90),ylower*steps:(ylower*steps+90)]

                #print("Boundary : X = ("+str(xlower*10)+", "+str(xlower*10+180)+")  
                #                        Y = ("+str(ylower*10)+", "+str(ylower*10+180)+")")

                boundary_box.append([xlower*steps, xlower*steps+90, ylower*steps, ylower*steps+90])

                X_data.append(temp)
                Y_data.append(0)



        X_data = np.array(X_data)
        print(X_data.shape)
        Y_data = np.array(Y_data)

        wid, hei = X_data.shape[1], X_data.shape[2]

        X_test = X_data.reshape(X_data.shape[0], 1, wid, hei).astype('float64')
        X_test = torch.from_numpy(X_test)

        Y_test = Y_data.reshape(Y_data.shape[0], 1).astype('float64')
        Y_test = torch.from_numpy(Y_test)

        print("Testing samples : "+str(X_test.shape[0]))

        test_data = TensorDataset(X_test, Y_test)

        testloader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)


        print("Prediction ... ")

        jj = 0



        with torch.no_grad():
            for data in testloader:

                images, y = data
                images, y = images.to(device), y.to(device)
                coutputs = class_model(images)
                #print(coutputs[0][0].round().detach().cpu().numpy())
                temp_pred.append(coutputs.detach().cpu().numpy())

                if coutputs[0][0].round().detach().cpu().numpy() == 1:
                    pred_box_loc.append(boundary_box[i])
                    pred_box_val.append(coutputs[0][0].detach().cpu().numpy())
                    #print(coutputs[0][0].detach().cpu().numpy(),boundary_box[i])

                if jj%10000==0:
                    print("Event : "+str(jj)+" Completed")

                jj+=1
                i+=1
            #print("xlower = "+str(xlower))
        print("Prediction done")

        temp_pred = np.concatenate(temp_pred)
        #print(temp_pred)

        all_pred.append(temp_pred.T[0])
        #print(np.array(pred_outputs).shape)
        #print(np.array(pred_box_val).shape)
        #print(np.array(pred_box_loc).shape)


    all_pred = np.array(all_pred)
    sampled_sig = np.array(sampled_sig)

    #np.save("diff_stepsize/500_pure_bkg_allpred_"+str(steps)+"step",all_pred)
    np.save("5000bkg_result/500_bkgp_g"+str(g_rescale)+"_n"+str(num_HS)+"_allpred_"+str(steps)+"step_"+str(bkg_number),all_pred)
    np.save("5000bkg_result/500_sampled_sig_g"+str(g_rescale)+"_n"+str(num_HS)+"_allpred_"+str(steps)+"step_"+str(bkg_number),sampled_sig)

(720, 720)
630 630


Event number = 1

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 2

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 3

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 4

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 5

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 6

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 7

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 8

(16129, 9

Prediction done


Event number = 61

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 62

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 63

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 64

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 65

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 66

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 67

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 68

(161

Event : 10000 Completed
Prediction done


Event number = 121

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 122

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 123

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 124

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 125

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 126

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 127

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 181

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 182

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 183

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 184

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 185

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 186

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 187

(16129, 90, 90)
Testing samples : 

Event : 10000 Completed
Prediction done


Event number = 240

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 241

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 242

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 243

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 244

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 245

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 246

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 300

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 301

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 302

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 303

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 304

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 305

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 306

(16129, 90, 90)
Testing samples : 

Event : 10000 Completed
Prediction done


Event number = 359

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 360

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 361

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 362

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 363

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 364

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 365

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction

In [2]:
# Implant PHS information

num_HS = 1
g_rescale = 0

In [3]:
# Load Sky map
path="/afs/crc.nd.edu/user/t/tkim12/Work/CMB_ML/Data/Nside1024/quicklens_720x720/"
pathsig = '/afs/crc.nd.edu/user/t/tkim12/Work/CMB_ML/Data/Nside1024/test_img/'

#bkg_only = np.empty((0, 720, 720))
sig_only = np.empty((0, 720, 720))

#bkg_number = 1

for bkg_number in range(8,10):
    
    bkg_only = np.load(path+"500_events_720Sqr_lensed_uncutBackgroundlmax3500_"+str(bkg_number)+".npy")


    for evnts in range(8):
        temp2 = np.load(pathsig+"sig_larger/200_PHS_720sq_g1_Sig_onLSS_"+str(evnts+1)+".npy")
        sig_only = np.concatenate((sig_only,temp2))

    steps = 5

    Xlowermax = int((bkg_only[0].T.shape[0]-90)/steps)*steps
    Ylowermax = int((bkg_only[0].T.shape[1]-90)/steps)*steps

    print(bkg_only[0].shape)
    print(Xlowermax,Ylowermax)

    all_pred = []
    sampled_sig = []

    for evnt_num in range(bkg_only.shape[0]):
    #for evnt_num in range(2):

        print("\n\nEvent number = "+str(evnt_num+1)+"\n")

        if g_rescale != 0:
            sampling = torch.randperm(len(sig_only))[:num_HS].numpy()
            sampled_sig.append(sampling)

            sigs = np.zeros((bkg_only[0].shape[0],bkg_only[0].shape[1]))

            for ii in range(len(sampling)):
                sigs += sig_only[sampling[ii]]

            tot = bkg_only[evnt_num]+g_rescale*sigs

        elif g_rescale == 0:
            tot = bkg_only[evnt_num]

        i=0
        pred_outputs = []
        pred_box_loc = []
        pred_box_val = []
        boundary_box = []

        temp_pred = []
        X_data = []
        Y_data = []

        for xlower in range(int(Xlowermax/steps+1)):

            for ylower in range(int(Ylowermax/steps+1)):

                temp = (tot)[xlower*steps:(xlower*steps+90),ylower*steps:(ylower*steps+90)]

                #print("Boundary : X = ("+str(xlower*10)+", "+str(xlower*10+180)+")  
                #                        Y = ("+str(ylower*10)+", "+str(ylower*10+180)+")")

                boundary_box.append([xlower*steps, xlower*steps+90, ylower*steps, ylower*steps+90])

                X_data.append(temp)
                Y_data.append(0)



        X_data = np.array(X_data)
        print(X_data.shape)
        Y_data = np.array(Y_data)

        wid, hei = X_data.shape[1], X_data.shape[2]

        X_test = X_data.reshape(X_data.shape[0], 1, wid, hei).astype('float64')
        X_test = torch.from_numpy(X_test)

        Y_test = Y_data.reshape(Y_data.shape[0], 1).astype('float64')
        Y_test = torch.from_numpy(Y_test)

        print("Testing samples : "+str(X_test.shape[0]))

        test_data = TensorDataset(X_test, Y_test)

        testloader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)


        print("Prediction ... ")

        jj = 0



        with torch.no_grad():
            for data in testloader:

                images, y = data
                images, y = images.to(device), y.to(device)
                coutputs = class_model(images)
                #print(coutputs[0][0].round().detach().cpu().numpy())
                temp_pred.append(coutputs.detach().cpu().numpy())

                if coutputs[0][0].round().detach().cpu().numpy() == 1:
                    pred_box_loc.append(boundary_box[i])
                    pred_box_val.append(coutputs[0][0].detach().cpu().numpy())
                    #print(coutputs[0][0].detach().cpu().numpy(),boundary_box[i])

                if jj%10000==0:
                    print("Event : "+str(jj)+" Completed")

                jj+=1
                i+=1
            #print("xlower = "+str(xlower))
        print("Prediction done")

        temp_pred = np.concatenate(temp_pred)
        #print(temp_pred)

        all_pred.append(temp_pred.T[0])
        #print(np.array(pred_outputs).shape)
        #print(np.array(pred_box_val).shape)
        #print(np.array(pred_box_loc).shape)


    all_pred = np.array(all_pred)
    sampled_sig = np.array(sampled_sig)

    #np.save("diff_stepsize/500_pure_bkg_allpred_"+str(steps)+"step",all_pred)
    np.save("5000bkg_result/500_bkgp_g"+str(g_rescale)+"_n"+str(num_HS)+"_allpred_"+str(steps)+"step_"+str(bkg_number),all_pred)
    np.save("5000bkg_result/500_sampled_sig_g"+str(g_rescale)+"_n"+str(num_HS)+"_allpred_"+str(steps)+"step_"+str(bkg_number),sampled_sig)

(720, 720)
630 630


Event number = 1

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 2

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 3

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 4

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 5

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 6

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 7

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 8

(16129, 9

Prediction done


Event number = 61

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 62

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 63

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 64

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 65

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 66

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 67

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 68

(161

Event : 10000 Completed
Prediction done


Event number = 121

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 122

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 123

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 124

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 125

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 126

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 127

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 181

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 182

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 183

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 184

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 185

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 186

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 187

(16129, 90, 90)
Testing samples : 

Event : 10000 Completed
Prediction done


Event number = 240

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 241

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 242

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 243

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 244

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 245

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 246

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction

...

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 217

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 218

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 219

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 220

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 221

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 222

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 223

(16129, 90, 90)
Testing samples : 

Event : 10000 Completed
Prediction done


Event number = 276

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 277

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 278

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 279

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 280

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 281

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 282

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 336

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 337

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 338

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 339

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 340

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 341

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 342

(16129, 90, 90)
Testing samples : 

Event : 10000 Completed
Prediction done


Event number = 395

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 396

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 397

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 398

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 399

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 400

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 401

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 455

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 456

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 457

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 458

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 459

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 460

(16129, 90, 90)
Testing samples : 16129
Prediction ... 
Event : 0 Completed
Event : 10000 Completed
Prediction done


Event number = 461

(16129, 90, 90)
Testing samples : 